In [ ]:
import os
import numpy as np
import pylab as py
import matplotlib.pyplot as plt
from spisea import synthetic, evolution, atmospheres, reddening
from spisea.imf import imf, multiplicity
from matplotlib.colors import LogNorm
from matplotlib.cm import ScalarMappable
import csv

# Paths for isochrones
iso_dir = 'isochrones/'

# Estimation variables
star_index = 0  # Star in the CSV to analyze

# Define isochrone parameters
dist = 4500
evo_model = evolution.Baraffe15()
atm_func = atmospheres.get_merged_atmosphere
red_law = reddening.RedLawCardelli(3.1)
filt_list = ['jwst,F162M', 'jwst,F182M']
filters = ['m_jwst_F162M', 'm_jwst_F182M']
metallicity = 0
level_ages = np.linspace(1, 10, 19) * 1e6  # Age array
log_age_arr = np.log10(level_ages)

# Change this as necessary
AKs = 0.7

# Load sample magnitudes, skipping the header row
sample_mags = []
with open('../../s284-162-182.csv', mode='r') as file:
    csvFile = csv.reader(file)
    next(csvFile)  # Skip header row
    for lines in csvFile:
        sample_mags.append([float(x) for x in lines])

# Chi-square minimization function
def chi_square_reverse_model(iso_grid, sample_mags):
    chi_square_values = np.zeros((len(iso_grid), len(iso_grid[0].points)))
    for i, iso in enumerate(iso_grid):
        for j, star in enumerate(iso.points):
            chi_square = sum(((sample_mags[k] - star[filters[k]]) ** 2) / star[filters[k]] for k in range(len(sample_mags)))
            chi_square_values[i, j] = chi_square
    return chi_square_values

# Ensure directory exists
os.makedirs(iso_dir, exist_ok=True)

# Clear previous isochrones
for file in os.listdir(iso_dir):
    os.remove(os.path.join(iso_dir, file))

# Generate isochrone grid
instances = np.array([
    synthetic.IsochronePhot(log_age, AKs, dist, metallicity=metallicity,
                            evo_model=evo_model, atm_func=atm_func,
                            red_law=red_law, filters=filt_list,
                            iso_dir=iso_dir)
    for log_age in log_age_arr
])

# Compute chi-square for each point
chi_square_values = chi_square_reverse_model(instances, sample_mags[star_index])

# Extract mass and age grids
masses = np.array([[star['mass'] for star in iso.points] for iso in instances])
ages = np.tile(10 ** log_age_arr[:, np.newaxis], (1, masses.shape[1]))

# Plot single diagram
fig, ax = plt.subplots(figsize=(10, 8))
norm = LogNorm(vmin=max(np.min(chi_square_values), 1e-3), vmax=np.max(chi_square_values))
cmap = plt.get_cmap('coolwarm')

# Scatter plot with chi-square values
sc = ax.scatter(ages, masses, c=chi_square_values, cmap=cmap, norm=norm, s=50, edgecolor='k', linewidth=0.5)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('Chi-Squared Value')

# Apply logarithmic scale to y-axis
ax.set_yscale('log')

# Add labels, grid, and title
ax.set_xlabel('Age (years)')
ax.set_ylabel('Mass (M☉)')
ax.set_title(f'Age-Mass Diagram with Chi-Squared Coloring (AKs = {AKs})')
ax.grid(True, which='both', linestyle='--', linewidth=0.5)

plt.show()


/home/wyz5rge/.local/lib/python3.9/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in /scratch/wyz5rge/models/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


Isochrone generation took 1.285532 s.
Making photometry for isochrone: log(t) = 6.00  AKs = 0.70  dist = 4500
     Starting at:  2025-01-29 06:06:26.679109   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F162M = 23.11
Starting filter: jwst,F182M   Elapsed time: 0.61 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F182M = 23.10
      Time taken: 1.21 seconds
Isochrone generation took 1.134837 s.
Making photometry for isochrone: log(t) = 6.18  AKs = 0.70  dist = 4500
     Starting at:  2025-01-29 06:06:29.044763   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2292 K  m_jwst_F162M = 23.47
Starting filter: jwst,F182M   Elapsed time: 0.61 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2292 K  m_jwst_F182M = 23.49
      Time taken: 1.22 seconds
Isochrone genera